In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing required libraries
from langchain import PromptTemplate #for templating
from langchain.chains import RetrievalQA #for retrieving ans
from langchain.embeddings import HuggingFaceEmbeddings #for creating embeddings
from langchain.vectorstores import Pinecone #for storing vectordb
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader #for pdf and dir loading
from langchain.text_splitter import RecursiveCharacterTextSplitter #for creating chunks
from langchain.prompts import PromptTemplate #for prompting
from langchain.llms import CTransformers #quantized model

In [3]:
#setting up pinecone database
PINECONE_API_KEY="34ce10a9-bb10-4fbb-854c-7ae3799d8d07"
PINECONE_API_ENV="gcp-starter"

In [4]:
#loading data
#Extracting data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf", #loads all pdf files from thd directory
                    loader_cls=PyPDFLoader) #lodar class is PyPDFLoader to load the pdfs
    documents = loader.load()
    return documents


In [5]:
pwd()

'/home/jovyan/work'

In [6]:
extracted_data = load_pdf("/home/jovyan/work/data/")

In [7]:
# extracted_data

In [8]:
#Creating text chunks from extracted data
#creating custom function to create chunk from input data
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [9]:
#creating chunks using our custom function
text_chunks = text_split(extracted_data=extracted_data)
print("length of my chunks: ",len(text_chunks))

length of my chunks:  7020


In [17]:
# text_chunks 

In [11]:
#Converting chunks to Vector Embeddings
#Creating custom function to download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
#download the embedding model
embeddings = download_hugging_face_embeddings()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
#testing/querying from embeddings
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result)) #o/p is vector embedding of Hello world

Length 384


In [15]:
#Initializing the Pinecone
#This method is deprecated and not working anymore
'''pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)'''
#use below method to initialize Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")

In [23]:
#Creating Embeddings for Each of The Text Chunks & storing
#This method is deprecated and not working anymore
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [24]:
#Creating Embeddings for Each of The Text Chunks & storing
# Insert data and embeddings
'''
for text_chunk, embedding in zip(text_chunks, embeddings):
    index.upsert(ids=[text_chunk.page_content], vectors=[embedding])'''

'\nfor text_chunk, embedding in zip(text_chunks, embeddings):\n    index.upsert(ids=[text_chunk.page_content], vectors=[embedding])'

In [19]:
from langchain.vectorstores import Chroma

In [20]:
persist_directory = 'db'

In [22]:
vectordb = Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory=persist_directory)
vectordb.persist()

ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help![0m